# Split PDF Horizontally or Vertically

* Author: docai-incubator@google.com

## Disclaimer

This tool is not supported by the Google engineering team or product team. It is provided and supported on a best-effort basis by the **DocAI Incubator Team**. No guarantees of performance are implied.

# Objective
This tool splits given pdf either horizontally or vertically and in the given percentage based on the configuration.

**Approach**: Each page of input pdf is first converted to image, these images are split as per the configuration and the final output pdf contains all the split pages of the input pdf.


# Prerequisites
* Vertex AI Notebook
* GCS Folder Path
* DocumentAI Parsed JSONs

# Step-by-Step Procedure

## 1. Import Modules/Packages

In [ ]:
import io
from typing import List

import pymupdf as fitz  # PyMuPDF
from PIL import Image

## 2. Input Details

* **INPUT_FILE** : Input file name which needs to split.
* **OUTPUT_PATH** : Provide directory path to store splitted pdf.

In [ ]:
INPUT_FILE = "sample_file_name.pdf"
OUTPUT_PATH = "./output_dir/path"

## 3. Run Below Code-Cells

In [ ]:
def convert_pdf_to_images(pdf_path: str, dpi: int = 300) -> List[Image.Image]:
    """Helper function to converts pdf to images

    Args:
        pdf_path (str): Locan  path to input pdf
        dpi (int, optional): dpi decides the quality of images. Defaults to 300.

    Returns:
        List[Image.Image]: List of PIL image objects
    """

    doc = fitz.open(pdf_path)
    images = []
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        # Create a pixmap with a specified DPI
        pix = page.get_pixmap(matrix=fitz.Matrix(dpi / 72, dpi / 72))
        img = Image.open(io.BytesIO(pix.tobytes()))
        images.append(img)
    return images


def split_image(
    image: Image.Image, split_type: str = "vertical", split_percent: int = 50
) -> List[Image.Image]:
    """It will split image either horizontally or vertically based on split_percent value

    Args:
        image (Image.Image): PIL Image object
        split_type (str, optional): Its value either horizontal or vertical. Defaults to "vertical".
        split_percent (int, optional): cent: Positive number n, such that 1<=n<=99, Defaults to 50.
            It decides where split has to happen. 50 splits the page into equal halves.

    Returns:
        List[Image.Image]: Returns list of cropped images
    """

    width, height = image.size
    if split_type == "horizontal":
        split_line = int(height * split_percent / 100)
        top_half = image.crop((0, 0, width, split_line))
        bottom_half = image.crop((0, split_line, width, height))
        return [top_half, bottom_half]
    elif split_type == "vertical":
        split_line = int(width * split_percent / 100)
        left_half = image.crop((0, 0, split_line, height))
        right_half = image.crop((split_line, 0, width, height))
        return [left_half, right_half]


def process_pdf(
    input_pdf_path: str,
    output_path: str,
    split_type: str = "vertical",
    split_percent: int = 50,
) -> None:
    """Splits each page input pdf vertically/horizontally in the split_percent
    ratio and saves the result as a new pdf

    Args:
        input_pdf_path (str): Path to input pdf (supports both single page/multipage)
        output_path (str): Path to output folder where split pdf will be saved
        split_type (str, optional): Either "horizontal" or "vertical". Defaults to "vertical".
        split_percent (int, optional): Positive number n, such that 1<=n<=99, It decides where
            split has to happen. 50 splits the page into equal halves.Defaults to 50.
    """

    print(f"Processing {input_pdf_path}")

    images = convert_pdf_to_images(input_pdf_path)
    output_path = output_path.rstrip("/")
    fn = input_pdf_path.split("/")[-1].split(".")[0]
    fn = fn + "_split_" + split_type + ".pdf"
    output_pdf_path = output_path + "/" + fn
    split_images = []

    for image in images:
        split_images.extend(split_image(image, split_type, split_percent))
    split_images[0].save(
        output_pdf_path, save_all=True, append_images=split_images[1:], resolution=300.0
    )
    print(f"\tFile saved to {output_pdf_path}")


# split_type and split_percent are optional input parameters.
# split_type takes either “vertical” or “horizontal”. It defaults to “vertical”.
# split_percent takes positive value between 1 and 100 defaults to 50.
process_pdf(INPUT_FILE, OUTPUT_PATH, split_type="vertical", split_percent=50)

# 4. Output Details

Refer below images for splitted pdf results

<img src="./images/sample_input.png" height=800 width=1000 alt='input_image'> </img>  
<b>Sample Output 1</b>  
<img src="./images/sample_output_left.png" height=150 width=300 alt='output_image_1'></img>  
<b>Sample Output 2 </b>  
<img src="./images/sample_output_right.png" height=150 width=300 alt='output_image_2'></img>